In [9]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from sentence_transformers import CrossEncoder
from langchain.schema import SystemMessage, HumanMessage, AIMessage
import gradio as gr

In [10]:
# This will setup teh OPENAI API key in os environment variables 
#os.environ["OPENAI_API_KEY"] = "xxxxx"
def setup_environment():
    import sys
    sys.path.append('C:\\gitworkspace\\aimldemo\\jupyterworkapce')
    import stratup_env_setup
    stratup_env_setup.set_env()

In [11]:
setup_environment()

Set env key: AWS_ACCESS_KEY_ID
Set env key: AWS_SECRET_ACCESS_KEY
Set env key: OPENAI_API_KEY
Set env key: HUGGINGFACEHUB_API_TOKEN
Set env key: LANGCHAIN_API_KEY
Set env key: API_KEY


In [8]:
import openai
if not openai.api_key:
    print("Error: OpenAI API key is missing.")
else:
    print("OpenAI API key is set.")

OpenAI API key is set.


In [4]:
# Load PDF document and create chunks
loader = PyPDFLoader("1728286846_the_nestle_hr_policy_pdf_2012.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [5]:
# Generate embeddings and store in Chroma DB
embedding = OpenAIEmbeddings()
vector_store = Chroma.from_documents(chunks, embedding=embedding)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_40596\1043584726.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


In [6]:
# Cross-Encoder for re-ranking
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [7]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-3.5-turbo")

C:\Users\jimmy\AppData\Local\Temp\ipykernel_40596\335954310.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo")


In [8]:
def chat_with_history(query, history):
    # Retrieve top n chunks from the vector database
    retriever = vector_store.as_retriever(search_kwargs={"k": 5})
    retrieved_docs = retriever.invoke(query)
    retrieved_texts = [doc.page_content for doc in retrieved_docs]

    # Cross-encode and rank to get the most relevant chunk
    query_pairs = [[query, text] for text in retrieved_texts]
    scores = cross_encoder.predict(query_pairs)
    best_match_index = scores.argmax()
    best_chunk = retrieved_texts[best_match_index]

    # Check if relevant context is found
    if scores[best_match_index] < 0.5:
        response = "I do not have enough information to answer your question."
        history.append(("User 👤", query))
        history.append(("Chatbot 🤖", response))
        return history

    # Construct conversation history
    conversation = [
        SystemMessage(content="You are a helpful assistant. Answer concisely using only the provided context and chat history.")
    ]
    for user_query, bot_response in history:
        conversation.append(HumanMessage(content=user_query))
        conversation.append(AIMessage(content=bot_response))
    conversation.append(HumanMessage(content=query))

    # Include retrieved context as additional system guidance
    conversation.append(SystemMessage(content=f"Context: {best_chunk}"))

    # Generate concise response with LLM
    response = llm.invoke(conversation).content.strip()

    # Update history with the user query and concise response
    history.append(("User 👤", query))
    history.append(("Chatbot 🤖", response))
    return history


In [9]:
def gradio_ui():
    # Chatbot UI starts with an empty history
    history = []

    def chat_interface(user_input, chat_history):
        chat_history = chat_with_history(user_input, chat_history)
        return chat_history, chat_history

    # Create Gradio Interface
    interface = gr.Blocks()

    with interface:
        gr.Markdown("<h1 style='text-align: center;'>Ask me anything about Nestlé HR Policy</h1>")
        chatbot = gr.Chatbot()
        user_input = gr.Textbox(label="Your Question", placeholder="Type your question here...")
        send_button = gr.Button("Send")
        close_button = gr.Button("Stop Server")  

        # Maintain chat history
        chat_history = gr.State([])

        # Button click triggers response generation
        send_button.click(
            fn=chat_interface,
            inputs=[user_input, chat_history],
            outputs=[chatbot, chat_history],
        )

        # Add functionality to close the Gradio server
        def stop_server():
            interface.close()
            print("Gradio server stopped.")

        close_button.click(fn=stop_server, inputs=[], outputs=[])

    interface.launch()

In [10]:
gradio_ui()

C:\Users\jimmy\AppData\Roaming\Python\Python311\site-packages\gradio\components\chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
